In [31]:
import os
import numpy as np

In [2]:
PATH_DIR = 'paths'

In [3]:
def stringToTuple(input_string):
    try:
        tuples_list = eval(input_string)
        
        if isinstance(tuples_list, list) and all(isinstance(t, tuple) and len(t) == 2 for t in tuples_list):
            return tuples_list
        else:
            raise ValueError("Invalid input format. Must be a list of tuples.")
    
    except Exception as e:
        raise ValueError("Error converting string to tuples: {}".format(str(e)))

def evalCost(input_string):
    try:
        # Using eval to parse the string into a Python expression
        list_of_lists = eval(input_string)
        
        # Check if the result is a list of lists
        if isinstance(list_of_lists, list) and all(isinstance(sublist, list) for sublist in list_of_lists):
            # Check if each element in the sublists is a float
            flattened_list = [elem for sublist in list_of_lists for elem in sublist]
            return list_of_lists
        else:
            raise ValueError("Invalid input format. Must be a list of lists.")
    
    except Exception as e:
        return None
        raise ValueError("Error converting string to list of lists: {}".format(str(e)))

In [4]:
dir_list = os.listdir(PATH_DIR)

paths = {'RRT' : {}, 'spike' : {}}
start_ends = []

for file in dir_list:
    if os.path.isdir(os.path.join(PATH_DIR, file)):
        continue
    comps = file[0:-4].split("_")
    
    pathtype = comps[0]
    
    start = comps[1].split("-")
    start = (int(start[0]), int(start[1]))

    end = comps[2].split("-")
    end = (int(end[0]), int(end[1]))

    point = ((start, end))

    with open(os.path.join(PATH_DIR, file)) as f:
        path = stringToTuple(f.readline())
        costs = evalCost(f.readline())
        if costs != None:
            paths[pathtype][point] = {'trajectory' : None, 'cost' : None}
            paths[pathtype][point]['trajectory'] = path
            paths[pathtype][point]['cost'] = costs

    if point not in start_ends and costs != None:
        start_ends.append(point)

In [5]:
def calculateCost(data):
    current_total = 0
    current_time = 0

    obstacle_total = 0
    obstacle_time = 0

    slope_total = 0
    slope_time = 0

    for wp in data:
        current_total += wp[1]
        current_time += wp[2]
        obstacle_total += wp[3]
        obstacle_time += wp[4]
        slope_total += wp[5]
        slope_time += wp[6]

    return [current_total, current_time, obstacle_total, obstacle_time, slope_total, slope_time]

In [6]:
rrts = []
swps = []

rrt_plens = []
swp_plens = []

for pts in start_ends:
    rrt_costs = paths['RRT'][pts]['cost']
    swp_costs = paths['spike'][pts]['cost']

    rrts.append(calculateCost(rrt_costs))
    swps.append(calculateCost(swp_costs))

    rrt_plens.append(len(paths['RRT'][pts]['trajectory']))
    swp_plens.append(len(paths['spike'][pts]['trajectory']))

if len(rrts) != len(swps):
    print("Number of data not the same")

In [11]:
import scipy.stats as stats

def two_sample_t_test(data1, data2, alpha=0.05):
    # Perform two-sample t-test
    t_statistic, p_value = stats.ttest_ind(data1, data2)

    # Degrees of freedom
    degrees_of_freedom = len(data1) + len(data2) - 2

    # Critical value for a two-tailed test
    critical_value = stats.t.ppf(1 - alpha / 2, degrees_of_freedom)

    # Determine if the null hypothesis should be rejected
    reject_null = abs(t_statistic) > critical_value

    result = {
        't_statistic': t_statistic,
        'p_value': p_value,
        'degrees_of_freedom': degrees_of_freedom,
        'critical_value': critical_value,
        'reject_null': reject_null
    }

    return result

In [29]:
swp_curr = [swps[i][1] / swps[i][0] for i in range(len(swps))]
rrt_curr = [rrts[i][1] / rrts[i][0] for i in range(len(rrts))]

swp_obs = [swps[i][2] for i in range(len(swps))]
rrt_obs = [rrts[i][2] for i in range(len(rrts))]

swp_slope = [swps[i][4] for i in range(len(swps))]
rrt_slope = [rrts[i][4] for i in range(len(rrts))]

In [33]:
print(np.mean(swp_slope))
print(np.mean(rrt_slope))

1331.5227082821696
1411.8941426803833


In [40]:
curr_result = two_sample_t_test(swp_curr, rrt_curr)
print(curr_result)

{'t_statistic': 0.25802187427833484, 'p_value': 0.7974084896406521, 'degrees_of_freedom': 52, 'critical_value': 2.0066468031022113, 'reject_null': False}


In [41]:
obs_result = two_sample_t_test(swp_obs, rrt_obs)
print(obs_result)

{'t_statistic': -1.8180164239000824, 'p_value': 0.07482411522388262, 'degrees_of_freedom': 52, 'critical_value': 2.0066468031022113, 'reject_null': False}


In [42]:
slope_result = two_sample_t_test(swp_slope, rrt_slope)
print(slope_result)

{'t_statistic': -0.3364577227319846, 'p_value': 0.7378797760089983, 'degrees_of_freedom': 52, 'critical_value': 2.0066468031022113, 'reject_null': False}


In [43]:
pathlen_result = two_sample_t_test(swp_plens, rrt_plens)
print(pathlen_result)

{'t_statistic': -1.116964691401176, 'p_value': 0.2691439059207824, 'degrees_of_freedom': 52, 'critical_value': 2.0066468031022113, 'reject_null': False}
